In [ ]:
import os
import json
import uuid
from glob import glob

from langchain.schema import Document
# from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
embedding_model = HuggingFaceEmbeddings(
    model_name="LazarusNLP/all-indobert-base-v2",
    model_kwargs={"device": "cuda"}   # ubah ke "cuda" jika pakai GPU
)

C:\Users\umar_\AppData\Local\Temp\ipykernel_43184\597545190.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
d:\sistem\anaconda\envs\klien-yolo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def save_to_chroma(documents, persist_dir="./chroma_db"):

    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_dir
    )

    vectordb.persist()
    print(f"[✔] Berhasil menyimpan {len(documents)} dokumen ke ChromaDB.")
    return vectordb

In [4]:
def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    for key, value in meta.items():
        # Jika LIST → ubah ke string koma
        if isinstance(value, list):
            cleaned[key] = ", ".join([str(v) for v in value])
        # Jika dict di dalam metadata → ubah ke string JSON
        elif isinstance(value, dict):
            cleaned[key] = json.dumps(value)
        # Jika tipe data primitif → simpan apa adanya
        elif isinstance(value, (str, int, float, bool)) or value is None:
            cleaned[key] = value
        else:
            # Jika tipe lain (tuple, set, object) → jadikan string
            cleaned[key] = str(value)
    return cleaned

In [5]:
def load_kb_files(kb_folder="./kb"):
    kb_files = glob(os.path.join(kb_folder, "*.json"))
    documents = []

    for file in kb_files:
        with open(file, "r", encoding="utf-8") as f:
            kb_data = json.load(f)

        # Pastikan file berisi list item KB
        if isinstance(kb_data, dict):
            kb_data = [kb_data]

        for item in kb_data:
            content = item.get("content", "")
            metadata = {
                "id": item.get("id", "unknown"),
                "category": item.get("category", "general"),
                "title": item.get("title", ""),
            }

            documents.append(Document(page_content=content, metadata=metadata))

    return documents

In [6]:
print("Memuat file KB...")
docs = load_kb_files("./data/json")

Memuat file KB...


In [7]:
print("Menyimpan ke ChromaDB…")
save_to_chroma(docs, persist_dir="./chroma_db")

Menyimpan ke ChromaDB…
[✔] Berhasil menyimpan 17 dokumen ke ChromaDB.


C:\Users\umar_\AppData\Local\Temp\ipykernel_43184\586489097.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
